# Load Balancing App
The Load Balancing CCO Application is a modular pipeline designed to train, evaluate, and utilize a Reinforcement Learning (RL) agent for dynamic Coverage and Capacity Optimization (CCO).

The primary goal is to create an intelligent agent that suggests optimal cell antenna tilt configurations for each hour of the day. The agent's policy is trained on multiple days of simulated UE traffic data to learn time-of-day patterns. Its objective is to find a balance between three key network KPIs:


In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import logging
import os
from IPython.display import Image, display

from apps.load_balancing.data_preprocessor import UEDataPreprocessor
from apps.load_balancing.bdt_manager import BDTManager
from apps.load_balancing.rl_trainer import run_rl_training
from apps.load_balancing.rl_predictor import run_rl_prediction
from apps.load_balancing.cco_visualizer import CCOVisualizer

In [ ]:
# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Setting up Data Directory & Save Locations
Unzip the `data/load_balancing_data.zip` file to get `data/load_balancing_data/` folder


In [ ]:
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

BASE_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "load_balancing_data", "generated_data")
STATIC_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "load_balancing_data", "data")

In [ ]:
# All relevant data required for this demo are located in the following directories
TOPOLOGY_PATH = os.path.join(STATIC_DATA_DIR, "topology.csv")
CONFIG_PATH = os.path.join(STATIC_DATA_DIR, "config.csv")
TRAINING_DATA_PATH = os.path.join(STATIC_DATA_DIR, "dummy_ue_training_data.csv")

# All data saved during the training and prediction processes are located in the following directories
BDT_MODEL_PATH = os.path.join("data", "load_balancing_data", "bdt_model_map.pickle")
RL_MODEL_PATH = os.path.join("data", "load_balancing_data", "load_balancing_agent.zip")
RL_LOG_DIR = os.path.join("data", "load_balancing_data", "rl_training_logs")
PLOT_OUTPUT_DIR = os.path.join(STATIC_DATA_DIR, "plots")

# Showcasing **Energy Savings** Functions: 

Define the ``TRAIN_DAYS`` and ``test_day`` days, **optionally** ``TOTAL_TIMESTEPS``

In [ ]:
TRAIN_DAYS = [0, 1, 2, 3]
TEST_DAY = 4
days_to_process = sorted(set(TRAIN_DAYS + [TEST_DAY]))
TOTAL_TIMESTEPS = 30000

# Set inference tick
TICK = 20

### 1. Preprocessing all `User Equipement` **(UE)** Data
Prepare raw, multi-day UE location data for use in the RL environment

In [ ]:
# 500 UEs x 24 ticks x 5 days = 60000 total data points
logger.info(f"--- Running UE Data Preprocessing for days: {days_to_process} ---")
preprocessor = UEDataPreprocessor(base_data_dir=BASE_DATA_DIR)
preprocessor.run(days=days_to_process)
logger.info("--- Preprocessing Step Finished ---")

### 2. Training core `bayesian_digital_twins` **(BDT)**
Train Bayesian Digital Twin model, in addition to downloading the resulting model

In [ ]:
# Set parameters
BDT_MODEL_ID = "bdt_for_load_balancing"
CONTAINER_NAME = "radp_dev-training-1"

In [ ]:
# Train bayesian_digital_twins from scratch
logger.info("--- Running BDT Training ---")
bdt_manager = BDTManager(
    topology_path=TOPOLOGY_PATH,
    training_data_path=TRAINING_DATA_PATH,
    model_path=BDT_MODEL_PATH
)
bdt_manager.train(model_id=BDT_MODEL_ID, container_name=CONTAINER_NAME)
logger.info("--- BDT Training Finished ---")

### 3. Train `RL Agent` for **Load Balancing** on The First 4 Days
Orchestrates the training of the PPO agent

In [ ]:
# Intialize and train RL agent
logger.info(f"--- Running RL Training on Days: {TRAIN_DAYS} ---")
run_rl_training(
    bdt_model_path=BDT_MODEL_PATH,
    base_ue_data_dir=BASE_DATA_DIR,
    training_days=TRAIN_DAYS,
    topology_path=TOPOLOGY_PATH,
    config_path=CONFIG_PATH,
    rl_model_path=RL_MODEL_PATH,
    log_dir=RL_LOG_DIR,
    total_timesteps=TOTAL_TIMESTEPS
)
logger.info("--- RL Training Finished ---")

### 4. Inference
Uses the trained RL agent to get an immediate recommendation for a specific hour

In [ ]:
# Run inference on exising RL model
logger.info(f"--- Running Inference for Tick {TICK} ---")
run_rl_prediction(
    model_load_path=RL_MODEL_PATH,
    topology_path=TOPOLOGY_PATH,
    target_tick=TICK
)
logger.info("--- Inference Step Finished ---")

### 5. Visualization
Perform the visualization of network performance & compare baseline configuration to an RL-optimized configuration.


In [ ]:
# Generate visualization plots
logger.info(f"--- Running Visualization for Day {TEST_DAY}, Tick {TICK} ---")
try:
    visualizer = CCOVisualizer(
        bdt_model_path=BDT_MODEL_PATH,
        rl_model_path=RL_MODEL_PATH,
        topology_path=TOPOLOGY_PATH,
        config_path=CONFIG_PATH,
        base_ue_data_dir=BASE_DATA_DIR
    )
    
    # Generate and save comparison plots for the baseline and RL-optimized scenarios.
    visualizer.generate_comparison_plots(
        day=TEST_DAY,
        tick=TICK,
        output_dir=PLOT_OUTPUT_DIR
    )
    logger.info("--- Visualization Step Finished ---")
except Exception as e:
    logger.exception(f"Visualization failed with error: {e}")

In [ ]:
image_path = os.path.join(PLOT_OUTPUT_DIR, f"cco_comparison_day{TEST_DAY}_tick_{TICK}.png")

# Display plot image
display(Image(filename=image_path))